In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
from statsmodels.stats.weightstats import ztest
from scipy.stats import chi2_contingency

In [2]:
df = pd.read_csv('../MachineLearningRating_v3.txt',delimiter='|')
print(df.head(10))

/tmp/ipykernel_7171/2311303730.py:1: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../MachineLearningRating_v3.txt',delimiter='|')


   UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered  \
0               145249     12827  2015-03-01 00:00:00             True   
1               145249     12827  2015-05-01 00:00:00             True   
2               145249     12827  2015-07-01 00:00:00             True   
3               145255     12827  2015-05-01 00:00:00             True   
4               145255     12827  2015-07-01 00:00:00             True   
5               145247     12827  2015-01-01 00:00:00             True   
6               145247     12827  2015-04-01 00:00:00             True   
7               145247     12827  2015-06-01 00:00:00             True   
8               145247     12827  2015-08-01 00:00:00             True   
9               145245     12827  2015-03-01 00:00:00             True   

  Citizenship          LegalType Title Language                 Bank  \
0              Close Corporation    Mr  English  First National Bank   
1              Close Corporation    Mr  E

In [3]:
# Convert TotalPremium and TotalClaims to numeric
df['TotalPremium'] = pd.to_numeric(df['TotalPremium'], errors='coerce')
df['TotalClaims'] = pd.to_numeric(df['TotalClaims'], errors='coerce')

In [6]:
# Define functions for statistical tests
# Define the chi-squared test function
def chi_squared_test(data, col1, col2):
    contingency_table = pd.crosstab(data[col1], data[col2])
    chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
    return chi2, p

# Define the t-test function
def t_test(data, group_col, value_col):
    group_a = data[data[group_col] == data[group_col].unique()[0]][value_col]
    group_b = data[data[group_col] == data[group_col].unique()[1]][value_col]
    t_stat, p = stats.ttest_ind(group_a, group_b, nan_policy='omit')
    return t_stat, p

In [8]:
# Null Hypothesis 1: There are no risk differences across provinces
print("Testing risk differences across provinces...")
chi2, p = chi_squared_test(df, 'Province', 'TotalClaims')
print(f"Chi-squared test: chi2={chi2}, p-value={p}")

Testing risk differences across provinces...
Chi-squared test: chi2=10722.684705166641, p-value=1.0


In [10]:
# Null Hypothesis 2: There are no risk differences between zip codes
print("Testing risk differences between zip codes...")
chi2, p = chi_squared_test(df, 'PostalCode', 'TotalClaims')
print(f"Chi-squared test: chi2={chi2}, p-value={p}")

Testing risk differences between zip codes...
Chi-squared test: chi2=1214174.406374286, p-value=1.0


In [11]:
# Null Hypothesis 3: There are no significant margin (profit) differences between zip codes
print("Testing profit margin differences between zip codes...")
df['ProfitMargin'] = df['TotalPremium'] - df['TotalClaims']
t_stat, p = t_test(df, 'PostalCode', 'ProfitMargin')
print(f"T-test: t_stat={t_stat}, p-value={p}")


Testing profit margin differences between zip codes...
T-test: t_stat=-0.4863459356321961, p-value=0.6268739198863356


In [12]:
# Null Hypothesis 4: There are no significant risk differences between Women and Men
print("Testing risk differences between Women and Men...")
chi2, p = chi_squared_test(df, 'Gender', 'TotalClaims')
print(f"Chi-squared test: chi2={chi2}, p-value={p}")

Testing risk differences between Women and Men...
Chi-squared test: chi2=2511.4776608232637, p-value=1.0


In [13]:
# Analyze and report results
def analyze_results(p_value, alpha=0.05):
    if p_value < alpha:
        return "Reject the null hypothesis"
    else:
        return "Fail to reject the null hypothesis"

In [14]:
# Analyzing and reporting
print("Results:")
print("Risk differences across provinces:", analyze_results(p))
print("Risk differences between zip codes:", analyze_results(p))
print("Profit margin differences between zip codes:", analyze_results(p))
print("Risk differences between Women and Men:", analyze_results(p))

Results:
Risk differences across provinces: Fail to reject the null hypothesis
Risk differences between zip codes: Fail to reject the null hypothesis
Profit margin differences between zip codes: Fail to reject the null hypothesis
Risk differences between Women and Men: Fail to reject the null hypothesis
